In [1]:
import utils as utils
from nltk.classify import NaiveBayesClassifier

In [2]:
DATA_FILE_PATH = 'data.json'

In [3]:
train_tups, test_tups = utils.split_data(utils.generate_tuples_from_file(DATA_FILE_PATH, num_samples=10000), test_size=0.2)

print('Training set size: ', len(train_tups[0]))
print('Test set size: ', len(test_tups[0]))
print('Sample tokenized review: ', train_tups[0][0])
print('Sample rating: ', train_tups[1][0])

Training set size:  8000
Test set size:  2000
Sample tokenized review:  ['Great', 'atmosphere', '.', 'And', 'great', 'burger', 'the', 'Fondu', 'burger', 'was', 'rediculous', '.', 'The', 'servers', 'are', "n't", 'bad', 'to', 'look', 'at', 'either', '.']
Sample rating:  5


In [4]:
def word_feats(list_of_words) -> dict:    
    return dict([(word, True) for word in list_of_words])

def word_feats_mult(list_of_words) -> dict:
    return dict([(word, list_of_words.count(word)) for word in list_of_words])

In [5]:
labeled_feature_sets = []
for i in range(len(train_tups[0])):
    feature_set = word_feats(train_tups[0][i])
    _class = train_tups[1][i]
    labeled_feature_sets.append((feature_set, _class))

classifier = NaiveBayesClassifier.train(labeled_featuresets=labeled_feature_sets)

In [6]:
unseen_review = word_feats('The food was absolutely terrible. I would never eat there.')
rating_prediction = classifier.classify(unseen_review)

print('Rating prediction: ', rating_prediction)

Rating prediction:  1
